In [ ]:
# --- Κελί 1: Mount + Αντιγραφή από Google Drive ---
from google.colab import drive
import os

drive.mount('/content/drive')
!rm -f /content/data/new_images/*.tif
!rm -f /content/data/new_images/*.tfw
# Δημιουργία φακέλων
# !mkdir -p /content/data/new_images

# Αντιγραφή GeoTIFF (.tif) και world files (.tfw) από Drive σε local
!cp /content/drive/MyDrive/road_extraction/new_images/*.tif /content/data/new_images/
!cp /content/drive/MyDrive/road_extraction/new_images/*.tfw /content/data/new_images/
!ls -lh /content/data/new_images


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 772M
-rw------- 1 root root   44 Jul 16 15:51 010_0302041145.tfw
-rw------- 1 root root 258M Jul 16 15:51 010_0302041145.tif
-rw------- 1 root root   44 Jul 16 15:51 012_0304041070.tfw
-rw------- 1 root root 258M Jul 16 15:51 012_0304041070.tif
-rw------- 1 root root   44 Jul 16 15:51 013_0304041085.tfw
-rw------- 1 root root 258M Jul 16 15:51 013_0304041085.tif


In [ ]:
!pip install rasterio
import cv2
from tqdm import tqdm
import numpy as np
import rasterio
from PIL import Image
from tensorflow.keras.models import load_model

# Φόρτωση μοντέλου (προσαρμόστε custom_objects αν χρειάζεται)
class IoUMetric:
    def __call__(self, y_true, y_pred):
        import tensorflow.keras.backend as K
        y_pred = K.cast(y_pred > 0.5, 'float32')
        intersection = K.sum(y_true * y_pred)
        union = K.sum(y_true) + K.sum(y_pred) - intersection
        return intersection / (union + K.epsilon())

model = load_model('/content/drive/MyDrive/road_segmentation_model.h5',
                   custom_objects={'iou_metric': IoUMetric()})
# --- 1. Μεμονωμένη εικόνα ---
def segment_large_geotiff_georeferenced(model, image_path, output_path,
                                        tile_size=256, overlap=0, threshold=0.3):
    with rasterio.open(image_path) as src:
        profile = src.profile
        transform = src.transform
        crs = src.crs
        img_np = src.read()  # (C, H, W)
        img_np = np.transpose(img_np, (1, 2, 0))  # (H, W, C)

    if img_np.shape[2] > 3:
        img_np = img_np[:, :, :3]
    if img_np.ndim == 2:
        img_np = np.stack([img_np]*3, axis=-1)

    h, w, c = img_np.shape
    mask_full = np.zeros((h, w), dtype=np.uint8)
    step = tile_size - overlap

    for y in range(0, h, step):
        for x in range(0, w, step):
            x_end = min(x + tile_size, w)
            y_end = min(y + tile_size, h)
            x_start = x_end - tile_size
            y_start = y_end - tile_size
            if x_start < 0: x_start = 0
            if y_start < 0: y_start = 0

            tile = img_np[y_start:y_start+tile_size, x_start:x_start+tile_size]
            tile_input = tile / 255.0
            tile_input = np.expand_dims(tile_input, axis=0)

            pred = model.predict(tile_input,verbose=0)[0]
            pred_mask = (pred.squeeze() > threshold).astype("uint8")
            kernel = np.ones((5, 5), np.uint8)
            pred_mask = cv2.morphologyEx(pred_mask, cv2.MORPH_CLOSE, kernel)

            mask_full[y_start:y_start+tile_size, x_start:x_start+tile_size] = np.maximum(
                mask_full[y_start:y_start+tile_size, x_start:x_start+tile_size], pred_mask
            )

    profile.update({'count': 1, 'dtype': 'uint8', 'compress': 'lzw'})
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(mask_full, 1)

    return output_path

# --- 2. Επεξεργασία φακέλου ---
def batch_segment_geotiff_folder(model, input_folder, output_folder,
                                  tile_size=256, overlap=0, threshold=0.3):
    os.makedirs(output_folder, exist_ok=True)
    tif_files = sorted([
        f for f in os.listdir(input_folder)
        if f.lower().endswith('.tif') and not f.lower().endswith('.tfw')
    ])

    for tif_file in tqdm(tif_files, desc="🔄 Επεξεργασία εικόνων"):
        input_path = os.path.join(input_folder, tif_file)
        output_path = os.path.join(output_folder, f"mask_{tif_file}")
        segment_large_geotiff_georeferenced(model, input_path, output_path,
                                            tile_size=tile_size,
                                            overlap=32,
                                            threshold=threshold)
        print(f"✅ Αποθηκεύτηκε: {output_path}")

    print("✅ Ολοκληρώθηκε η επεξεργασία όλων των εικόνων.")

# Εκτέλεση batch
batch_segment_geotiff_folder(
    model=model,
    input_folder="/content/data/new_images",
    output_folder="/content/data/predicted_masks",
    tile_size=256,
    overlap=0,
    threshold=0.3
)


🔄 Επεξεργασία εικόνων:  33%|███▎      | 1/3 [01:16<02:33, 76.70s/it]

✅ Αποθηκεύτηκε: /content/data/predicted_masks/mask_010_0302041145.tif


🔄 Επεξεργασία εικόνων:  67%|██████▋   | 2/3 [02:31<01:15, 75.54s/it]

✅ Αποθηκεύτηκε: /content/data/predicted_masks/mask_012_0304041070.tif


🔄 Επεξεργασία εικόνων: 100%|██████████| 3/3 [03:46<00:00, 75.46s/it]

✅ Αποθηκεύτηκε: /content/data/predicted_masks/mask_013_0304041085.tif
✅ Ολοκληρώθηκε η επεξεργασία όλων των εικόνων.


In [ ]:
# --- Κελί 3: Zip + Επιστροφή στο Google Drive ---
!zip -r /content/predicted_masks.zip /content/data/predicted_masks
!cp /content/predicted_masks.zip /content/drive/MyDrive/road_extraction/predicted_masks.zip


updating: content/data/predicted_masks/ (stored 0%)
updating: content/data/predicted_masks/mask_010_0302041145.tif (deflated 61%)
updating: content/data/predicted_masks/mask_012_0304041070.tif (deflated 49%)
updating: content/data/predicted_masks/mask_013_0304041085.tif (deflated 57%)
